# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 
import time
from IPython.display import display

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
display(df.head(5))
display(df.tail(5))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


In [4]:
df.duplicated().sum()

0

In [5]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Дубликатов нет

## Разбейте данные на выборки

Подготовим данные к классификации сохраним признаки в переменной features, а целевой признак в переменной target

In [6]:
features = df.drop(['is_ultra', 'calls'],axis=1)
target = df['is_ultra']

Так как нужно создать сначала тестовую выборку применим метод train_test_split дважды:
* Разделим данные на 0.8 и 0.2
* Разделим получившиеся данные на 0.25 и 0.75 
В результате получим правильное разделение данных на обучающие 0.6 и валидационные 0.2 и тестовые 0.2

Получим тестовую выборку:

In [7]:
features_80, features_test, target_80, target_test = train_test_split(
    features, target, test_size=0.2, random_state=2034)

Получим валидационную выборку и обучающие данные:

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_80, target_80, test_size=0.25, random_state=2034)

Теперь есть три набора данных обучающая выборка, валидационная и тестовая.

## Исследуйте модели

**Дерево решений:**

Напишим функцию которая будет менять глубину дерева от 1  до 10 и выберет лучшую модель дерева решений на основе Accuarry:

In [9]:
model_tree = None
best_result_1 = 0

for depth in range(1, 11):
    model_1 = DecisionTreeClassifier(random_state=2034, max_depth=depth)  
    model_1.fit(features_train,target_train) 
    result_1 = model_1.score(features_valid,target_valid) 
    if result_1 > best_result_1:
        model_tree = model_1
        best_result_1 = result_1

In [10]:
display("Accuracy наилучшей модели на валидационной выборке:", best_result_1)

'Accuracy наилучшей модели на валидационной выборке:'

0.8055987558320373

Необходимая точность получена

**Логистическая регрессия:**

In [11]:
model_log = LogisticRegression(random_state=2034, solver='lbfgs')
model_log.fit(features_train, target_train)
result_2 = model_log.score(features_valid, target_valid)
display("Accuracy  модели логистической регрессии на валидационной выборке:", result_2)

'Accuracy  модели логистической регрессии на валидационной выборке:'

0.7060653188180405

Необходимая точность  не получена

**Cлучайный лес:**

In [12]:
model_forest = None
best_result = 0
for est in range(1, 51):
    model = RandomForestClassifier(random_state=2034, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_valid,target_valid) 
    if result > best_result:
        model_forest = model
        best_result = result

In [13]:
display("Accuracy наилучшей модели на валидационной выборке:", best_result)

'Accuracy наилучшей модели на валидационной выборке:'

0.8102643856920684

Необходимая точность получена

В двух моделях мы получили необходимую точность, нужно выбрать одну, поэтому сравних их по времени работы

In [14]:
model_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=44,
                       n_jobs=None, oob_score=False, random_state=2034,
                       verbose=0, warm_start=False)

In [15]:
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=2034, splitter='best')

In [16]:
start_time = time.time()
model_forest.predict(features_valid)
display("Время выполнения решения model_forest %s секунд" % (time.time() - start_time))

'Время выполнения решения model_forest 0.013231754302978516 секунд'

In [17]:
start_time = time.time()
model_tree.predict(features_valid)
display("Время выполнения решения model_tree %s секунд" % (time.time() - start_time))

'Время выполнения решения model_tree 0.0017881393432617188 секунд'

**Вывод**


Так как точность решения у двух поделей отличаются минимально, определим лучшую по времени решения, а это модель **model_tree**

## Проверьте модель на тестовой выборке

**Проверим качество модели на тестовой выборке:**


Пололучим предсказания по тестовой выборке:

In [18]:
predictions_test = model_tree.predict(features_test)

Получим предсказания по обучающей выборке:

In [19]:
predictions_train = model_tree.predict(features_train)

Получим точность модели на обучающей выборке:

In [20]:
accuarry_train = accuracy_score(target_train, predictions_train)

Получим точность модели на тестовой выборке:

In [21]:
accuarry_test = accuracy_score(target_test, predictions_test)

In [22]:
display('Accuracy')
display('Обучающая выборка:', accuarry_train)
display('Тестовая выборка:', accuarry_test)

'Accuracy'

'Обучающая выборка:'

0.7956431535269709

'Тестовая выборка:'

0.7947122861586314

## Проверьте модели на адекватность

Посмотрим какой самый часто встречающейся класс в тестовой выборке:

In [23]:
target_test.value_counts()

0    444
1    199
Name: is_ultra, dtype: int64

Передадим моделе данные состоящие только из 0 и сделаем проверку

In [24]:
predictions_adequacy = [0]*target_test.count()
adequacy_test = accuracy_score(target_test, predictions_adequacy)

In [25]:
display('Accuracy')
display('Accurac при одинаковых ответах:', adequacy_test)
display('Тестовая выборка:', accuarry_test)

'Accuracy'

'Accurac при одинаковых ответах:'

0.6905132192846034

'Тестовая выборка:'

0.7947122861586314

Судя по результатом выше, модель действительно обучилась.

## Вывод

Необходимая для решения поставленой задачи модель выбрана, она удовлетворяет требованию - доля правильных ответов должна быть не ниже 0.75, на тестовой выборке модель показала точность 0.8 , и имеет оптимальную скорость решения. Модель проверена на вменяемость - она действительно обучилась.